In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor, PrintMonitor
from mand.core import find, addFootnote
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
class C(Entity):
    
    @node
    def fib(self, n):
        if n < 2:
            return n
        return self.fib(n-1) + self.fib(n-2)


In [3]:
c = C()

In [4]:
with SummaryMonitor():
    v = c.fib(10)
print v

with SummaryMonitor():
    v = c.fib(40)
print v

# Compute activity summary (0.00 seconds of wall clock time)

|key|value|
|-|-|
|GetValue|19
|GetValue/Calc|11

55


# Compute activity summary (0.01 seconds of wall clock time)

|key|value|
|-|-|
|GetValue|61
|GetValue/Calc|30

102334155


In [5]:
class D(Entity):
    
    @node(tweakable=True)
    def v(self): 
        return 10
        
class E(D):
    
    @node(tweakable=True)
    def v(self): 
        return super(E, self).v() + 20
    
e = E()

print 'Expect 30:', e.v()
assert e.v() == 30

with Context({e.v: 40}):
    print 'Expect 40:', e.v()
    assert e.v() == 40
        
print 'Expect 30:', e.v()
assert e.v() == 30

with Context({super(E, e).v: 40}):
    print 'Expect 60:', e.v()
    assert e.v() == 60

Expect 30: 30
Expect 40: 40
Expect 30: 30
Expect 60: 60


In [6]:
class R(Entity):

    @node
    def clocks(self):
        clock = self.getObj(_tr.RootClock, 'Main')
    
        def fn(node):
            return True
        
        nodes = find(clock.cutoffs, fn)
        return nodes
       
    @node 
    def clocks2(self):
        with Context({}):
            return self.clocks()
        
with db:
    r = R()
print r.clocks()
for c in r.clocks():
    print c, c.tweakable

for c in r.clocks2():
    print c, c.tweakable

    
with Context({}):
    for c in r.clocks():
        print c, c.tweakable

[<Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root @4555679056 T=True>]
<Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root @4555679056 T=True> True
<Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4477767576 @4556482256 T=True> True
<Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4477767576 @4556483728 T=True> True
